In [4]:
import pandas as pd
from zipfile import ZipFile
from bs4 import BeautifulSoup
import requests
import re, webbrowser
from urllib.request import urlopen
from io import BytesIO
import datetime 
import time

### Getting Data from the Web

In [41]:
### Read file from BAG
link = "https://www.covid19.admin.ch/api/data/20210209-zyp7ir0s/downloads/sources-csv.zip"
resp = urlopen(link)
zipfile = ZipFile(BytesIO(resp.read()))

#### VaccDosesAdministered

In [42]:
fname = zipfile.namelist()[-4]
df_bag = pd.read_csv(zipfile.open(fname), dtype=object)

In [43]:
fname

'data/COVID19VaccDosesAdministered.csv'

In [8]:
### Save BAG file local
TodaysDate = time.strftime("%d-%m-%Y")
excelfilename = "COVID19VaccDosesAdministered_" + TodaysDate +".xlsx"
excelfilename
df_bag.to_excel(r'BAG\\' + excelfilename)

In [44]:
### Prepare file for merge
df_bag.drop(df_bag.index[[-1,-2]], inplace = True)
df_bag = df_bag.rename(columns = {"sumTotal": "Total_Impfungen", "date": "Stand", "per100PersonsTotal": "Impfungen pro 100 Einwohner", "geoRegion": "Abk."})
df_bag["Stand"] = pd.to_datetime(df_bag["Stand"])
df_bag["Impfungen pro 100 Einwohner"] = pd.to_numeric(df_bag["Impfungen pro 100 Einwohner"]).astype(float)
df_bag["Total_Impfungen"] = pd.to_numeric(df_bag["Total_Impfungen"]).astype("int64")
df_bag.drop(columns = ["type", "pop","version"], inplace= True)

In [40]:
# Transform old data --> one time
df_vaccines = pd.read_excel("impf_daten.xlsx")
df_vaccines.head(10)
#df_vaccines= df_vaccines.sort_values(by = ["Abk.", "Stand"])
#df_vaccines= df_vaccines.fillna(method='ffill')
#df_vaccines["Impfungen pro 100 Einwohner"] = round(df_vaccines["Impfungen pro 100 Einwohner"], 1)

,Unnamed: 0,Unnamed: 0.1,Kanton,Abk.,Total_Impfungen,Einwohner,Impfungen pro 100 Einwohner,Stand,lat,lng
0,0,0,Aargau,AG,0,685845,0.0,2020-12-21,47.3923,8.0446
1,1,1,Aargau,AG,12100,685845,1.8,2021-01-13,47.3923,8.0446
2,2,2,Aargau,AG,12973,685845,1.9,2021-01-24,47.3923,8.0446
3,3,3,Aargau,AG,16730,685845,2.4,2021-01-27,47.3923,8.0446
4,4,4,Aargau,AG,20034,685845,2.9,2021-01-31,47.3923,8.0446
5,5,10,Aargau,AG,24812,685845,3.6,2021-02-03,47.3923,8.0446
6,6,6,Appenzell-I.,AI,0,16128,0.0,2020-12-21,47.3162,9.4317
7,7,7,Appenzell-I.,AI,40,16128,0.2,2020-12-24,47.3162,9.4317
8,8,8,Appenzell-I.,AI,823,16128,5.1,2021-01-15,47.3162,9.4317
9,9,9,Appenzell-I.,AI,917,16128,5.7,2021-01-24,47.3162,9.4317


In [31]:
# Merge data and remove duplicates
m =pd.concat([df_vaccines, df_bag], axis = 0)
d = m.sort_values(by = ["Abk.", "Stand"])
dfcombined = d.fillna(method='ffill')
dfcombined["Impfungen pro 100 Einwohner"] = pd.to_numeric(dfcombined["Impfungen pro 100 Einwohner"]).fillna(0).astype(float)
dfcombined["Impfungen pro 100 Einwohner"] = round(dfcombined["Impfungen pro 100 Einwohner"], 1)
dfcombined.reset_index(drop=True, inplace=True)
dfcombined =dfcombined.drop_duplicates()


In [32]:
dfcombined.head(20)

,Unnamed: 0,Kanton,Abk.,Total_Impfungen,Einwohner,Impfungen pro 100 Einwohner,Stand,lat,lng
0,0,Aargau,AG,0,685845.0,0.0,2020-12-21,47.3923,8.0446
1,1,Aargau,AG,12100,685845.0,1.8,2021-01-13,47.3923,8.0446
2,2,Aargau,AG,12973,685845.0,1.9,2021-01-24,47.3923,8.0446
3,3,Aargau,AG,16730,685845.0,2.4,2021-01-27,47.3923,8.0446
4,4,Aargau,AG,20034,685845.0,2.9,2021-01-31,47.3923,8.0446
5,10,Aargau,AG,24812,685845.0,3.6,2021-02-03,47.3923,8.0446
6,6,Appenzell-I.,AI,0,16128.0,0.0,2020-12-21,47.3162,9.4317
7,7,Appenzell-I.,AI,40,16128.0,0.2,2020-12-24,47.3162,9.4317
8,8,Appenzell-I.,AI,823,16128.0,5.1,2021-01-15,47.3162,9.4317
9,9,Appenzell-I.,AI,917,16128.0,5.7,2021-01-24,47.3162,9.4317


In [38]:
#Write impf data
dfcombined.to_excel("impf_daten.xlsx") 

#### VaccDosesDelivered

In [13]:
fname = zipfile.namelist()[-3]
df_deliv = pd.read_csv(zipfile.open(fname), dtype=object)
fname

'data/COVID19VaccDosesDelivered.csv'

In [20]:
df_deliv

,geoRegion,date,pop,sumTotal,per100PersonsTotal,type,version
0,GE,2021-01-31,504128,28050,5.56,COVID19VaccDosesDelivered,2021-01-31_18-00-00
1,VD,2021-01-31,805098,43425,5.39,COVID19VaccDosesDelivered,2021-01-31_18-00-00
2,VS,2021-01-31,345525,20975,6.07,COVID19VaccDosesDelivered,2021-01-31_18-00-00
3,FR,2021-01-31,321783,17325,5.38,COVID19VaccDosesDelivered,2021-01-31_18-00-00
4,NE,2021-01-31,176496,12975,7.35,COVID19VaccDosesDelivered,2021-01-31_18-00-00
5,JU,2021-01-31,73584,5700,7.75,COVID19VaccDosesDelivered,2021-01-31_18-00-00
6,BE,2021-01-31,1039474,63050,6.07,COVID19VaccDosesDelivered,2021-01-31_18-00-00
7,SO,2021-01-31,275247,17325,6.29,COVID19VaccDosesDelivered,2021-01-31_18-00-00
8,BS,2021-01-31,195844,14550,7.43,COVID19VaccDosesDelivered,2021-01-31_18-00-00
9,BL,2021-01-31,289468,21150,7.31,COVID19VaccDosesDelivered,2021-01-31_18-00-00


#### Owid data

In [23]:
df_owid = pd.read_csv("https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/country_data/Switzerland.csv")

In [24]:
df_owid

,location,date,vaccine,source_url,total_vaccinations,people_vaccinated,people_fully_vaccinated
0,Switzerland,2020-12-23,Pfizer/BioNTech,https://www.nytimes.com/2020/12/23/world/switz...,0,NaN,NaN
1,Switzerland,2021-01-14,Pfizer/BioNTech,https://twitter.com/srfnews/status/13497272413...,66000,NaN,NaN
2,Switzerland,2021-01-19,Pfizer/BioNTech,https://www.srf.ch/news/schweiz/das-neueste-zu...,110000,NaN,NaN
3,Switzerland,2021-01-21,Pfizer/BioNTech,https://www.bag.admin.ch/dam/bag/fr/dokumente/...,169783,NaN,NaN
4,Switzerland,2021-01-24,Pfizer/BioNTech,https://www.bag.admin.ch/dam/bag/fr/dokumente/...,197368,NaN,NaN
5,Switzerland,2021-01-29,"Moderna, Pfizer/BioNTech",https://www.bag.admin.ch/dam/bag/fr/dokumente/...,262081,NaN,NaN
6,Switzerland,2021-01-31,"Moderna, Pfizer/BioNTech",https://www.covid19.admin.ch/en/epidemiologic/...,315033,NaN,NaN
